In [1]:
#BiocManager::install("ChIPseeker")

# load diffbind
library(DiffBind)

library(tidyverse)

library(ChIPseeker)
library(ggplot2)
library(TxDb.Hsapiens.UCSC.hg38.knownGene) 
txdb <- TxDb.Hsapiens.UCSC.hg38.knownGene

# for filtering ranges
BiocManager::install('plyranges')
library(plyranges)

Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb

Loa

## load sample sheet

In [2]:
setwd("/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1")

#sample_csv = "/wynton/group/ahituv/data/US-MPRA/ATAC-seq/diffbind_samples/samples_diffbind_red.csv"

results_path = "/wynton/group/ahituv/fongsl/projects/EMF/results/ATAC/WTC11NGN2.1"
sample_csv = "/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/samples_diffbind_wtc11.csv"
sample_csv1 = "/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/samples_diffbind_wtc11_maskctrl1.csv"
samples <- read.csv(sample_csv)

samples

SampleID,Tissue,Factor,Replicate,Peaks,PeakCaller,bamReads
<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>
WTC11.NGN2.2.ctrl1,WTC11.NGN2,control,1,/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/results/bwa/merged_library/macs2/broad_peak/control_REP1.mLb.clN_peaks.broadPeak,broad,/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/results/bwa/merged_library/control_REP1.mLb.clN.sorted.bam
WTC11.NGN2.2.ctrl3,WTC11.NGN2,control,2,/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/results/bwa/merged_library/macs2/broad_peak/control_REP2.mLb.clN_peaks.broadPeak,broad,/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/results/bwa/merged_library/control_REP2.mLb.clN.sorted.bam
WTC11.NGN2.2.ctrl4,WTC11.NGN2,control,3,/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/results/bwa/merged_library/macs2/broad_peak/control_REP3.mLb.clN_peaks.broadPeak,broad,/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/results/bwa/merged_library/control_REP3.mLb.clN.sorted.bam
WTC11.NGN2.2.PEMF1,WTC11.NGN2,treatment,1,/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/results/bwa/merged_library/macs2/broad_peak/PEMF_REP1.mLb.clN_peaks.broadPeak,broad,/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/results/bwa/merged_library/PEMF_REP1.mLb.clN.sorted.bam
WTC11.NGN2.2.PEMF2,WTC11.NGN2,treatment,2,/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/results/bwa/merged_library/macs2/broad_peak/PEMF_REP2.mLb.clN_peaks.broadPeak,broad,/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/results/bwa/merged_library/PEMF_REP2.mLb.clN.sorted.bam
WTC11.NGN2.2.PEMF3,WTC11.NGN2,treatment,3,/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/results/bwa/merged_library/macs2/broad_peak/PEMF_REP3.mLb.clN_peaks.broadPeak,broad,/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/results/bwa/merged_library/PEMF_REP3.mLb.clN.sorted.bam
WTC11.NGN2.2.PEMF4,WTC11.NGN2,treatment,4,/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/results/bwa/merged_library/macs2/broad_peak/PEMF_REP4.mLb.clN_peaks.broadPeak,broad,/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/results/bwa/merged_library/PEMF_REP4.mLb.clN.sorted.bam


## load object

In [3]:
obj = dba(sampleSheet=sample_csv, scoreCol=5,  # if broadpeak, scoreCol is 5 - https://support.bioconductor.org/p/9151399/
          config = data.frame(AnalysisMethod=DBA_DESEQ2) 
         )
obj

WTC11.NGN2.2.ctrl1 WTC11.NGN2 control   1 broad

WTC11.NGN2.2.ctrl3 WTC11.NGN2 control   2 broad

WTC11.NGN2.2.ctrl4 WTC11.NGN2 control   3 broad

WTC11.NGN2.2.PEMF1 WTC11.NGN2 treatment   1 broad

WTC11.NGN2.2.PEMF2 WTC11.NGN2 treatment   2 broad

WTC11.NGN2.2.PEMF3 WTC11.NGN2 treatment   3 broad

WTC11.NGN2.2.PEMF4 WTC11.NGN2 treatment   4 broad



7 Samples, 133126 sites in matrix (199485 total):
                  ID     Tissue    Factor Replicate Intervals
1 WTC11.NGN2.2.ctrl1 WTC11.NGN2   control         1     89580
2 WTC11.NGN2.2.ctrl3 WTC11.NGN2   control         2    117371
3 WTC11.NGN2.2.ctrl4 WTC11.NGN2   control         3    125424
4 WTC11.NGN2.2.PEMF1 WTC11.NGN2 treatment         1    116128
5 WTC11.NGN2.2.PEMF2 WTC11.NGN2 treatment         2    113990
6 WTC11.NGN2.2.PEMF3 WTC11.NGN2 treatment         3    108875
7 WTC11.NGN2.2.PEMF4 WTC11.NGN2 treatment         4    110978

## Make read count affinity matrix 
- requires bam or external count file

In [4]:
obj <- dba.count(obj, score=DBA_SCORE_RPKM_FOLD)

obj

Computing summits...

Re-centering peaks...



7 Samples, 132722 sites in matrix:
                  ID     Tissue    Factor Replicate    Reads FRiP
1 WTC11.NGN2.2.ctrl1 WTC11.NGN2   control         1 18318017 0.22
2 WTC11.NGN2.2.ctrl3 WTC11.NGN2   control         2 20077834 0.28
3 WTC11.NGN2.2.ctrl4 WTC11.NGN2   control         3 24572668 0.29
4 WTC11.NGN2.2.PEMF1 WTC11.NGN2 treatment         1 16289116 0.37
5 WTC11.NGN2.2.PEMF2 WTC11.NGN2 treatment         2 16704248 0.42
6 WTC11.NGN2.2.PEMF3 WTC11.NGN2 treatment         3 15638662 0.43
7 WTC11.NGN2.2.PEMF4 WTC11.NGN2 treatment         4 19713445 0.38

## correlate count scores

In [5]:
# correlation based on count scores, 
jpeg(file = file.path(results_path,  "2_count.jpg"), 
    width=600, height=600)

plot(obj)

dev.off()

png 
  2

## Constrast setup

In [6]:
# set up contrast for differential accessibility analysis
obj_contrast <- dba.contrast(obj,
                              categories=DBA_FACTOR,
                              minMembers=3)  # default

Computing results names...



## DESeq2 run

In [7]:
# do differential accessibility analysis - generates consensus peak set, read counting, normalization, differentially accessible peaks using contrasts
obj_analyze <- dba.analyze(obj_contrast,
                            method=DBA_ALL_METHODS)  # do both edger and deseg2

Applying Blacklist/Greylists...

Genome detected: Hsapiens.UCSC.hg38

Applying blacklist...

Removed: 28 of 132722 intervals.

Removed 28 (of 132722) consensus peaks.

Normalize edgeR with defaults...

Normalize DESeq2 with defaults...

Analyzing...



## report differentially accessible peaks from DESeq2

In [8]:
obj.deseq2 <- dba.report(obj_analyze,
                          contrast=1,
                          method=DBA_DESEQ2)

# write
write.csv(data.frame(obj.deseq2), file=paste(results_path, "deseq2.csv", sep='/'), row.names=FALSE)

# report number of sig differentially accessible peaks, fc >1, fc<1
sum(obj.deseq2$FDR<0.05)
sum(obj.deseq2$Fold>1)
sum(obj.deseq2$Fold<-1)

[1] 40299

[1] 2

[1] 1

## write all peaks, regardless of differentially bound (DB) or not (notDB)

In [9]:
obj.deseq2_nodb <- dba.report(obj_analyze,
                            contrast = 1,
                            th=1,
                            method = DBA_DESEQ2)

# write
write.csv(data.frame(obj.deseq2_nodb), file=paste(results_path,"deseq2_notDB.csv",sep='/'), row.names=FALSE)

sum(obj.deseq2_nodb$FDR>0.05)

[1] 92395

## filter chromosomes

In [10]:
data_GR = makeGRangesFromDataFrame(obj.deseq2)

data_GR

data_GR %>% 
  filter(seqnames == 'chr1')

GRanges object with 40299 ranges and 0 metadata columns:
                 seqnames              ranges strand
                    <Rle>           <IRanges>  <Rle>
  129633 chrUn_KI270438v1       111952-112352      *
  129631 chrUn_KI270438v1       103862-104262      *
  129632 chrUn_KI270438v1       109645-110045      *
  129637 chrUn_KI270467v1           2884-3284      *
   45307            chr16   46387732-46388132      *
     ...              ...                 ...    ...
   95726             chr5   34474514-34474914      *
  117425             chr8       697538-697938      *
  117335             chr7 158304172-158304572      *
  105688             chr6   71934469-71934869      *
   58626            chr19   58581142-58581542      *
  -------
  seqinfo: 39 sequences from an unspecified genome; no seqlengths

GRanges object with 4009 ranges and 0 metadata columns:
        seqnames              ranges strand
           <Rle>           <IRanges>  <Rle>
   1885     chr1   28369485-28369885      *
    506     chr1     7704444-7704844      *
   9282     chr1 202341816-202342216      *
   1821     chr1   27321974-27322374      *
   3365     chr1   52702894-52703294      *
    ...      ...                 ...    ...
   2333     chr1   34870409-34870809      *
  11357     chr1 240122890-240123290      *
   7461     chr1 163977836-163978236      *
   9507     chr1 204975647-204976047      *
  11680     chr1 246301565-246301965      *
  -------
  seqinfo: 39 sequences from an unspecified genome; no seqlengths

## plot heatmap

In [11]:
# heatmap
pdf(file = file.path(results_path,  "3_after_analyze.pdf"), width=8,height=8)
plot(obj_analyze)
dev.off()

png 
  2

## plot PCA

In [12]:
# PCA
pdf(file = file.path(results_path,  "4_pca.pdf"), width=8,height=8)
dba.plotPCA(obj_analyze)
dev.off()

png 
  2

## annotate peaks

library(TxDb.Hsapiens.UCSC.hg38.knownGene) 

In [13]:
txdb <- TxDb.Hsapiens.UCSC.hg38.knownGene

In [14]:
# for deseq2 sig peaks
peakAnno <- annotatePeak(obj.deseq2, 
                       tssRegion=c(-1000, 1000),
                       TxDb=txdb, 
                       annoDb="org.Hs.eg.db")

# write
write.csv(data.frame(peakAnno), file=paste(results_path,"deseq2_peakAnno.csv",sep='/'), row.names=FALSE)

>> preparing features information...		 2024-06-13 04:20:35 PM 
>> identifying nearest features...		 2024-06-13 04:20:37 PM 
>> calculating distance from peak to TSS...	 2024-06-13 04:20:38 PM 
>> assigning genomic annotation...		 2024-06-13 04:20:38 PM 
>> adding gene annotation...			 2024-06-13 04:21:25 PM 


Loading required package: org.Hs.eg.db



'select()' returned 1:many mapping between keys and columns



>> assigning chromosome lengths			 2024-06-13 04:21:27 PM 
>> done...					 2024-06-13 04:21:27 PM 


In [15]:
peakAnno <- annotatePeak(obj.deseq2_nodb, 
                       tssRegion=c(-1000, 1000),
                       TxDb=txdb, 
                       annoDb="org.Hs.eg.db")

# write
write.csv(data.frame(peakAnno), file=paste(results_path,"deseq2_notDB_peakAnno.csv",sep='/'), row.names=FALSE)

>> preparing features information...		 2024-06-13 04:21:28 PM 
>> identifying nearest features...		 2024-06-13 04:21:28 PM 
>> calculating distance from peak to TSS...	 2024-06-13 04:21:31 PM 
>> assigning genomic annotation...		 2024-06-13 04:21:31 PM 
>> adding gene annotation...			 2024-06-13 04:21:38 PM 


'select()' returned 1:many mapping between keys and columns



>> assigning chromosome lengths			 2024-06-13 04:21:39 PM 
>> done...					 2024-06-13 04:21:39 PM 


## plot annotations

In [17]:
pdf(file = file.path(results_path,  "5_peakAnno_all.pdf"), width=8,height=8)
plotAnnoBar(peakAnno)
dev.off()

png 
  2

# mask control 1

In [ ]:
obj = dba(sampleSheet=sample_csv1, scoreCol=5,  # if broadpeak, scoreCol is 5 - https://support.bioconductor.org/p/9151399/
          config = data.frame(AnalysisMethod=DBA_DESEQ2) 
         )
obj

obj <- dba.count(obj, score=DBA_SCORE_RPKM_FOLD)

obj

# correlation based on count scores, 
jpeg(file = file.path(results_path,  "2_count_no_ctrl1.jpg"), 
    width=600, height=600)

plot(obj)

dev.off()

# set up contrast for differential accessibility analysis
obj_contrast <- dba.contrast(obj,
                              categories=DBA_FACTOR,
                              minMembers=2)  # changed to accomodate 2 ctrl samples

# do differential accessibility analysis - generates consensus peak set, read counting, normalization, differentially accessible peaks using contrasts
obj_analyze <- dba.analyze(obj_contrast,
                            method=DBA_ALL_METHODS)  # do both edger and deseg2

obj.deseq2 <- dba.report(obj_analyze,
                          contrast=1,
                          method=DBA_DESEQ2)

# write
write.csv(data.frame(obj.deseq2), file=paste(results_path, "deseq2_noctrl1.csv", sep='/'), row.names=FALSE)

# report number of sig differentially accessible peaks, fc >1, fc<1
sum(obj.deseq2$FDR<0.05)
sum(obj.deseq2$Fold>1)
sum(obj.deseq2$Fold<-1)

# heatmap
pdf(file = file.path(results_path,  "3_after_analyze_noctrl.pdf"), width=8,height=8)
plot(obj_analyze)
dev.off()

# PCA
pdf(file = file.path(results_path,  "4_pca_noctrl.pdf"), width=8,height=8)
dba.plotPCA(obj_analyze)
dev.off()